In [12]:
import tushare as ts
from tushare.util import dateu as du
import pandas as pd
repo_dir = '/home/pi/practise/python_learning/repo'
#连绿6天的股票
registered_st = 'green_in6days_st'
days_count = 6
import os

#保存到CSV文件
if not os.path.isfile(repo_dir + '/' + du.last_tddate() + '.csv'):
    #一次性获取当前交易所有股票的行情数据（如果是节假日，即为上一交易日，结果显示速度取决于网速）
    df = ts.get_today_all()
    df.to_csv(repo_dir + '/' + du.last_tddate()+'.csv', encoding='utf-8')
else:
    df = pd.read_csv(repo_dir + '/' + du.last_tddate()+'.csv')
    

df_green_st = df[(df.changepercent < 0)]

#df_green_st.count()
    


In [17]:
#得到所有阴的股票
df_green_st.head(10)

,Unnamed: 0,code,name,changepercent,trade,open,high,low,settlement,volume,turnoverratio,amount,per,pb,mktcap,nmc
0,0,603999,读者传媒,-2.465,29.28,30.14,30.14,29.20,30.02,4020980,3.49043,118674144,52.286,5.200,843264.00000,337305.60000
1,1,603998,方盛制药,-0.984,18.12,18.27,18.41,18.04,18.30,3766387,1.39922,68558757,28.313,8.584,784608.17664,487749.12264
2,2,603997,继峰股份,-5.010,18.20,19.18,19.18,18.05,19.16,3834406,3.36351,70947110,41.364,5.251,764400.00000,207480.00000
5,5,603990,麦迪科技,-5.521,49.63,51.87,51.98,49.00,52.53,2780076,13.90038,139310021,76.354,15.131,397040.00000,99260.00000
6,6,603989,艾华集团,-4.513,34.70,36.10,36.19,34.00,36.34,1760278,2.08626,61143340,39.885,6.060,1041000.00000,292781.25000
7,7,603988,中电电机,-3.479,81.29,87.00,87.00,80.80,84.22,2152676,9.61016,181091290,153.377,10.055,650320.00000,182089.60000
8,8,603987,康德莱,-5.141,27.49,28.99,29.25,27.00,28.98,4731445,8.99514,132546523,45.817,6.530,578087.21000,144597.40000
10,10,603979,金诚信,-1.848,18.06,18.36,18.54,18.05,18.40,3077900,1.64553,56063041,28.667,2.222,812700.00000,337805.79840
11,11,603977,国泰集团,-0.197,25.29,25.30,25.44,24.52,25.34,4390259,7.94186,109480741,42.864,6.392,559111.32000,139803.12000
12,12,603969,银龙股份,-1.305,13.61,13.79,13.97,13.61,13.79,1367122,0.89571,18865951,41.242,3.516,544400.00000,207730.11050


In [3]:
#df_green_st[(df_green_st['code'] != '603979')].count()

code             2065
name             2065
changepercent    2065
trade            2065
open             2065
high             2065
low              2065
settlement       2065
volume           2065
turnoverratio    2065
amount           2065
per              2065
pb               2065
mktcap           2065
nmc              2065
dtype: int64

In [19]:
days_count = 7
target_list = []
#总跌幅
total_loss_list = []
for item in df_green_st['code']:
    tmpdf = ts.get_hist_data(str(item), start=du.day_last_week( 0 - days_count - 1))
    #打印出来看看
    print tmpdf
    #如果连续 days_count 天都阴
    try:
        if (tmpdf[(tmpdf.p_change < 0)])['p_change'].count() == days_count:
            target_list.append(item)
            #total_loss_list.append((item, tmpdf['p_change'].sum()))
            print 'item added:'+ str(item) + ', count:' +  str ((tmpdf[(tmpdf.p_change < 0)])['p_change'].count() )
        #else:
        #    print '.'
    except Exception:
        #ignore the error here
        pass
print 'Done'        

             open   high  close    low    volume  price_change  p_change  \
date                                                                       
2017-01-13  30.14  30.14  29.28  29.20  40209.80         -0.74     -2.46   
2017-01-12  30.01  30.39  30.02  29.79  32876.21          0.05      0.17   
2017-01-11  30.65  30.78  29.97  29.86  50476.20         -0.96     -3.10   
2017-01-10  31.22  31.27  30.93  30.70  60448.17         -0.53     -1.69   
2017-01-09  30.55  31.95  31.46  30.53  98014.73          1.05      3.45   
2017-01-06  30.34  30.50  30.41  30.05  42029.44          0.26      0.86   

               ma5    ma10    ma20     v_ma5    v_ma10    v_ma20  turnover  
date                                                                        
2017-01-13  30.332  30.211  30.877  56405.02  49880.19  62250.09      3.49  
2017-01-12  30.558  30.265  30.936  56768.95  51156.97  62682.79      2.85  
2017-01-11  30.584  30.300  30.934  58054.97  52019.85  63135.45      4.38  
2017-0

KeyboardInterrupt: 

In [10]:
#len(target_list)

#找出连跌的股，包括新股
df_target_tmp = df_green_st[(df_green_st.code.isin(target_list))]

In [11]:
newst = ts.new_stocks()
#看看个数
#newst['code'].count()

#上市日期150天以内的算新股（1年前或者几个月前的就不算新股了)
df_newst = newst[(newst.issue_date > du.day_last_week(-150)) | (newst.issue_date.isnull()) ]

#去掉新股，等到涨停的股
df_target = df_target_tmp[ ~(df_target_tmp.code.isin(df_newst.code))]


#重置下index的值，
df_target.index=range(df_target.code.count())

df_target

[Getting data:]###########

,Unnamed: 0,code,name,changepercent,trade,open,high,low,settlement,volume,turnoverratio,amount,per,pb,mktcap,nmc


In [58]:
df_target.ix[:,['code','name']]

,code,name
0,603518,维格娜丝
1,600572,康恩贝
2,600305,恒顺醋业
3,300202,聚龙股份
4,300200,高盟新材
5,300131,英唐智控


-9.8209999999999997

In [60]:
#df_target['changepercent'].sum()

#例子如何解包
a = ((1,2), (3, 4), (5, 6), (7, 8), (9, 10))
x,y = [[z[i] for z in a] for i in (0,1)]


from pandas import Series

mycodes,total_loss = [[z[i] for z in total_loss_list] for i in (0,1)]
df_final = df_target.ix[:,['code','name']]
#df_final['totalloss'] = Series(total_loss, index=df_final.index)

#df_final
total_loss_list

[(u'603987', -13.529999999999999),
 (u'603928', -17.530000000000001),
 (u'603900', -14.380000000000003),
 (u'603878', -26.73),
 (u'603658', -11.460000000000001),
 (u'603518', -10.329999999999998),
 (u'603323', -7.5),
 (u'603060', -16.91),
 (u'600572', -4.0000000000000009),
 (u'600305', -3.5599999999999996),
 (u'300575', -17.990000000000002),
 (u'300565', -22.34),
 (u'300202', -8.2400000000000002),
 (u'300200', -9.0800000000000018),
 (u'300131', -5.9500000000000002)]

In [45]:
#df_test = df_green_st[(df_green_st['code'] == '603979')]

In [54]:

#abc = ['test']
#df_test.loc['xxxx'] = Series(abc, index=df_test.index)
#df_test

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
  app.launch_new_instance()


,code,name,changepercent,trade,open,high,low,settlement,volume,turnoverratio,amount,per,pb,mktcap,nmc,xxxx
10,603979,金诚信,-1.393,18.4,18.7,18.76,18.38,18.66,2429627,1.29894,45111886,29.206,2.264,828000,344165.376,test
